<a href="https://colab.research.google.com/github/reedhodges/pytorch-loop-integrals/blob/main/pytorch_loop_integrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using PyTorch to classify loop integrals by the type of their divergence

You should run this notebook in Google Colab; it is configured to download the necessary files from the Github repository.  Just make sure you run everything in the 'Preliminaries' section below before beginning.

### Preliminaries

In [ ]:
import torch
import requests
import zipfile
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"

data_path = Path("data/")
image_path = data_path / "integrand_data"

if image_path.is_dir():
    print(f"[INFO] Directory {image_path} already exists, skipping download.")
else:
    print(f"[INFO] Creating {image_path} directory...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "integrand_data.zip", "wb") as f:
    url = "https://github.com/reedhodges/pytorch-loop-integrals/raw/main/integrand_data.zip"
    response = requests.get(url)
    print(f"[INFO] Downloading zip from {url}...")
    f.write(response.content)

with zipfile.ZipFile(data_path / "integrand_data.zip", "r") as zip_ref:
    print(f"[INFO] Extracting zip to {image_path}...")
    zip_ref.extractall(image_path)

engine_path = "engine.py"
utils_path = "utils.py"

if not Path(engine_path).is_file():
    print(f"[INFO] Downloading {engine_path}...")
    with open(engine_path, "wb") as f:
        url = "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/engine.py"
        response = requests.get(url)
        f.write(response.content)
else:
    print(f"[INFO] File {engine_path} already exists, skipping download.")

if not Path(utils_path).is_file():
    print(f"[INFO] Downloading {utils_path}...")
    with open(utils_path, "wb") as f:
        url = "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/utils.py"
        response = requests.get(url)
        f.write(response.content)
else:
    print(f"[INFO] File {utils_path} already exists, skipping download.")

print(f"[INFO] All files downloaded successfully.")

### Set up data and model

In [ ]:
train_path = image_path / "train"
test_path = image_path / "test"

image_path_list = list(data_path.glob("integrand_data/*/*/*.png"))

In [ ]:
from utils import fix_error_with_weights_download
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

fix_error_with_weights_download()

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights).to(device)

data_transform = weights.transforms()

for param in model.features.parameters():
    param.requires_grad = False

### Train

In [ ]:
import os
import torch
from torch import nn, optim
from engine import create_data_loaders, train

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
NUM_EPOCHS = 3
path_to_data = image_path / "data"

train_loader, test_loader, class_names, class_dict = create_data_loaders(path_to_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, train_transform=data_transform, test_transform=data_transform)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names)).to(device)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

 
train(device, model, train_loader, test_loader, loss_fn, optimizer, epochs=NUM_EPOCHS)